In [ ]:
"""
CS 484 Final Project

Alexander Wei
Kaiz Nanji
"""

In [1]:
from src.model import *
from src.utils import *

In [2]:
USE_GPU = False

In [4]:
"""
Uncomment the first time you are running to download the MNIST dataset
Once you've downloaded this, comment out this function
"""
# download()


(Dataset MNIST
     Number of datapoints: 60000
     Root location: ./data
     Split: Train,
 Dataset MNIST
     Number of datapoints: 10000
     Root location: ./data
     Split: Test)